In [ ]:
# main_rag.py
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from source.ingest.preprocessing import build_documents_from_xml
from langchain_upstage import ChatUpstage, UpstageEmbeddings

import re
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

# ---------- 1. 보험유형 정의 ----------
INSURANCE_TYPES = [
    "상해보험",
    "손해보험",
    "질병보험",
    "연금보험",
    "자동차보험",
    "책임보험",
    "화재보험"
]

# ---------- 2. LLM 초기화 ----------
llm_classifier = ChatUpstage(model="solar-1-mini-chat",temperature=0)
llm_qa = ChatUpstage(model="solar-1-mini-chat",temperature=0)

# ---------- 3. 보험유형 분류 함수 ----------
prompt_type_template = """
당신은 보험 상담 전문가입니다.
아래 질문을 보고 가장 관련 있는 보험 유형 하나를 선택하세요.
보험 유형: {types}

질문: {question}

출력 형식:
보험유형: <보험유형>
"""

prompt_type = PromptTemplate.from_template(prompt_type_template)

def classify_insurance(question: str) -> str:
    formatted_prompt = prompt_type.format(question=question, types=", ".join(INSURANCE_TYPES))
    raw = llm_classifier.invoke(formatted_prompt)
    result = str(raw)
    match = re.search(r"보험유형\s*:\s*(\S+)", result)
    if match:
        return match.group(1)
    else:
        return "알 수 없음"

# ---------- 4. 전처리된 문서 로드 ----------
# 기존 preprocessing.py로 생성된 문서 활용
all_docs = build_documents_from_xml("../data/001_상해보험_가공.xml")

# 유형별로 미리 분류 (예: 상해보험 / 손해보험...)
# 여기서는 단순 예시: 모든 문서가 상해보험이면 그냥 그대로 사용
insurance_docs = {typ: all_docs for typ in INSURANCE_TYPES}

# ---------- 5. 임베딩 & VectorStore ----------
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

vectorstores = {}
for typ, docs in insurance_docs.items():
    vectorstores[typ] = FAISS.from_documents(docs, embedding=embeddings)

# ---------- 6. Retriever + RAG ----------
def get_rag_answer(question: str) -> str:
    insurance_type = classify_insurance(question)
    print(f"분류된 보험유형: {insurance_type}")

    if insurance_type not in vectorstores:
        print("알 수 없는 보험유형, 전체 검색 실행")
        retriever = FAISS.from_documents(all_docs, embedding=embeddings).as_retriever()
    else:
        retriever = vectorstores[insurance_type].as_retriever()

    # 관련 문서 top-k 검색
    retrieved_docs = retriever.invoke(question)
    context_text = "\n\n".join(
        f"[{d.metadata['gwan']} {d.metadata['jo']}]\n{d.page_content}" for d in retrieved_docs
    )

    # QA Prompt
    prompt_qa_template = """
당신은 보험 약관 QA 전문가입니다.
아래 Context는 반드시 약관 원문입니다.

Context:
{context}

Question:
{question}

출력은 반드시 근거 조항을 포함하고, 한국어로 답변하십시오.
"""
    prompt_qa = PromptTemplate.from_template(prompt_qa_template)
    formatted_prompt = prompt_qa.format(context=context_text, question=question)

    # LLM QA 실행
    response = llm_qa.invoke(formatted_prompt)
    return str(response)



In [3]:

# ---------- 7. 테스트 ----------
if __name__ == "__main__":
    q1 = "버스에서 넘어져서 다리를 다쳐 입원했는데 보험 보장받을 수 있어?"
    q2 = "퇴직 후 연금 수령 조건이 궁금해요"

    for q in [q1, q2]:
        print("질문:", q)
        print(get_rag_answer(q))
        print("-" * 50)

질문: 버스에서 넘어져서 다리를 다쳐 입원했는데 보험 보장받을 수 있어?
분류된 보험유형: 상해보험\n\n상해보험은
알 수 없는 보험유형, 전체 검색 실행
content='버스에서 넘어져서 다리를 다쳐 입원한 경우, 보험 보장을 받을 수 있는지 여부는 몇 가지 조건에 따라 결정됩니다.\n\n1. 대중교통 이용 중 교통재해에 해당하는지 확인해야 합니다. [제2관 보험금의 지급 제4조 (대중교통 이용 중 교통재해의 정의)]에 따르면, 대중교통 이용 중 교통재해는 별표4(대중교통 이용 중 교통재해 분류표)에서 정한 교통사고를 말합니다. 따라서, 버스에서 넘어진 사고가 해당 분류표에 해당하는지 확인해야 합니다.\n\n2. 장해지급률에 해당하는지 확인해야 합니다. [제2관 보험금의 지급 제6조(보험금의 지급사유)]에 따르면, 보험금 지급사유 중 하나는 보험기간 중 피보험자가 대중교통 이용 중 교통재해를 직접적인 원인으로 장해분류표에서 정한 장해지급률 중 3 이상 100 이하에 해당하는 장해상태가 되었을 때입니다. 따라서, 다리를 다쳐 발생한 장해가 해당 장해지급률에 해당하는지 확인해야 합니다.\n\n3. 계약전 알릴의무를 위반하지 않았는지 확인해야 합니다. [제5관 보험료의 납입 제17조(계약전 알릴의무 위반의 효과)]에 따르면, 진단계약에서 보험금 지급사유가 발생할 때까지 진단을 받지 않은 경우라도 재해로 보험금 지급사유가 발생한 경우에는 보장을 해드립니다. 따라서, 다리를 다친 사고 이전에 진단을 받지 않았더라도, 해당 사고가 재해로 인한 것이라면 보험 보장을 받을 수 있습니다.\n\n위 조건들을 모두 만족한다면, 버스에서 넘어져서 다리를 다쳐 입원한 경우에도 보험 보장을 받을 수 있습니다. 그러나, 보험금 청구 서류를 위조 또는 변조한 경우에는 [제6관 계약의 해지 및 해지환급금 등 제32조(해지환급금)]에 따라 계약이 해지될 수 있으니 주의해야 합니다.' additional_kwargs={'refusal': None} response_metadata={'toke